<a href="https://colab.research.google.com/github/MestDash/PID/blob/main/notebooks/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions

In [11]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split
import os

def split_main_strat(
    df, main_label_col, sub_label_col, pcode_col,
    target_main_class="III: Predominantly antibody deficiencies",
    train_size=0.7, val_size=0.15, test_size=0.15,
    random_state=42,
    save_to_csv=False,
    output_dir='splits'
):
    assert abs(train_size + val_size + test_size - 1.0) < 1e-6, "Splits must sum to 1.0"

    # Automatically detect feature columns
    feature_columns = df.drop(columns=[main_label_col, sub_label_col, pcode_col]).columns.tolist()

    # Simplify sub-labels for the target main class
    def simplify_sub_label(row):
        if row[main_label_col] == target_main_class:
            first_char = str(row[sub_label_col]).strip()[0]
            return 'group_1to3' if first_char in {'1', '2', '3'} else 'group_4'
        else:
            return 'NA'

    df = df.copy()
    df['simplified_sub'] = df.apply(simplify_sub_label, axis=1)

    # Stratify only on main_label
    strat_labels = df[main_label_col]

    # First split: Train vs Temp
    df_train, df_temp = train_test_split(
        df,
        stratify=strat_labels,
        test_size=(1.0 - train_size),
        random_state=random_state
    )

    # Second split: Validation vs Test
    df_val, df_test = train_test_split(
        df_temp,
        stratify=df_temp[main_label_col],
        test_size=(test_size / (val_size + test_size)),
        random_state=random_state
    )

    # Identify metadata columns
    all_columns = set(df.columns)
    exclude_columns = set(feature_columns)
    meta_columns = list(all_columns - exclude_columns)

    # Helper to extract and optionally save splits
    def process_and_save_split(df_split, split_name):
        X = df_split[feature_columns].reset_index(drop=True)
        y = df_split[[main_label_col, 'simplified_sub']].reset_index(drop=True)
        meta = df_split[meta_columns].reset_index(drop=True)

        if save_to_csv:
            os.makedirs(output_dir, exist_ok=True)
            X.to_csv(os.path.join(output_dir, f'X_{split_name}.csv'), index=False)
            y.to_csv(os.path.join(output_dir, f'y_{split_name}.csv'), index=False)
            meta.to_csv(os.path.join(output_dir, f'meta_{split_name}.csv'), index=False)

        return {'X': X, 'y': y, 'meta': meta}

    return {
        'train': process_and_save_split(df_train, 'train'),
        'val': process_and_save_split(df_val, 'val'),
        'test': process_and_save_split(df_test, 'test')
    }


def split_hierarchical_strat(
    df, main_label_col, sub_label_col, pcode_col,
    train_size=0.7, val_size=0.15, test_size=0.15,
    random_state=42,
    save_to_csv=False,
    output_dir='splits'
):
    assert abs(train_size + val_size + test_size - 1.0) < 1e-6, "Splits must sum to 1.0"

    # Automatically detect feature columns
    feature_columns = df.drop(columns=[main_label_col, sub_label_col, pcode_col]).columns.tolist()

    # Create stratification key
    combined_strat = df[main_label_col].astype(str) + '__' + df[sub_label_col].astype(str)

    # Identify rare strata (with only 1 sample)
    strat_counts = combined_strat.value_counts()
    rare_mask = combined_strat.isin(strat_counts[strat_counts == 1].index)

    df_common = df[~rare_mask].copy()
    df_rare = df[rare_mask].copy()

    # First split: Train vs Temp
    df_train, df_temp = train_test_split(
        df_common,
        stratify=combined_strat[~rare_mask],
        test_size=(1.0 - train_size),
        random_state=random_state
    )

    # Second split: Val vs Test
    combined_temp_strat = df_temp[main_label_col].astype(str) + '__' + df_temp[sub_label_col].astype(str)
    relative_val_size = val_size / (val_size + test_size)
    df_val, df_test = train_test_split(
        df_temp,
        stratify=combined_temp_strat,
        test_size=(1.0 - relative_val_size),
        random_state=random_state
    )

    # Assign RARE (non-stratifiable) samples
    np.random.seed(random_state)
    rare_split_labels = np.random.choice(
        ['train', 'val', 'test'],
        size=len(df_rare),
        p=[train_size, val_size, test_size]
    )
    df_rare['split'] = rare_split_labels

    df_train = pd.concat([df_train, df_rare[df_rare['split'] == 'train'].drop(columns='split')], ignore_index=True)
    df_val   = pd.concat([df_val,   df_rare[df_rare['split'] == 'val'].drop(columns='split')], ignore_index=True)
    df_test  = pd.concat([df_test,  df_rare[df_rare['split'] == 'test'].drop(columns='split')], ignore_index=True)

    # Identify metadata columns
    all_columns = set(df.columns)
    exclude_columns = set(feature_columns)
    meta_columns = list(all_columns - exclude_columns)

    # Helper to process and optionally save splits
    def process_and_save_split(df_split, split_name):
        X = df_split[feature_columns].reset_index(drop=True)
        y = df_split[[main_label_col, sub_label_col]].reset_index(drop=True)
        meta = df_split[meta_columns].reset_index(drop=True)

        if save_to_csv:
            os.makedirs(output_dir, exist_ok=True)
            X.to_csv(os.path.join(output_dir, f'X_{split_name}.csv'), index=False)
            y.to_csv(os.path.join(output_dir, f'y_{split_name}.csv'), index=False)
            meta.to_csv(os.path.join(output_dir, f'meta_{split_name}.csv'), index=False)

        return {'X': X, 'y': y, 'meta': meta}

    return {
        'train': process_and_save_split(df_train, 'train'),
        'val': process_and_save_split(df_val, 'val'),
        'test': process_and_save_split(df_test, 'test')
    }


def count_label_combinations(df, main_label_col, sub_label_col):
    counts = (
        df.groupby([main_label_col, sub_label_col])
        .size()
        .reset_index(name='count')
        .sort_values(by=[main_label_col, 'count'], ascending=[True, True])
        .reset_index(drop=True)
    )
    return counts

def encode_iuis_columns(df):
    # Step 1: Map main classes
    iuis_class_map = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}
    df = df.copy()  # Optional: avoid modifying original DataFrame

    # Handle 'No arguments for lymphoid-PID'
    df["IUIS"] = df["IUIS"].astype(str)
    no_pid_mask = df["IUIS"] == "No arguments for lymphoid-PID"
    df.loc[no_pid_mask, "IUIS"] = 0
    df.loc[no_pid_mask, "IUIS extended"] = 0

    # Assign numeric class values
    mask_classified = ~no_pid_mask
    df.loc[mask_classified, "IUIS"] = (
        df.loc[mask_classified, "IUIS"]
        .apply(lambda x: iuis_class_map.get(str(x).split(":")[0], 0))
    )

    # Ensure type consistency
    df["IUIS"] = df["IUIS"].astype(int)

    # Step 2: Encode 'IUIS extended'
    df["IUIS extended"] = df["IUIS extended"].astype(str)
    df.loc[df["IUIS extended"] == "Unclassified", "IUIS extended"] = 0
    df["IUIS extended"] = df["IUIS extended"].apply(lambda x: int(str(x)[0]))

    # Step 3: Subclassification columns
    df["seq1"] = (df["IUIS"] == 1).astype(int)
    df["seq2"] = (df["IUIS"] != 0).astype(int)

    # Define seq3 logic
    def map_seq3(row):
        iuis = row["IUIS"]
        ext = row["IUIS extended"]
        if iuis == 0:
            return 5
        elif iuis == 1:
            return 4
        elif iuis == 2:
            return 0
        elif iuis == 3:
            return 2 if ext == 4 else 1
        elif iuis == 4:
            return 3
        return -1  # fallback

    df["seq3"] = df.apply(map_seq3, axis=1)

    return df


def stratified_split_and_save(
    df,
    non_feature_cols,
    label_col,
    output_dir="output_splits",
    train_size=0.6,
    val_size=0.2,
    test_size=0.2,
    random_state=17
):
    """
    Split a DataFrame into train, validation, and test sets with stratification,
    and save features, labels, and metadata for each set as CSV files.

    Parameters:
    - df: input DataFrame
    - non_feature_cols: list of columns to exclude from features
    - label_col: name of the label column (must be in non_feature_cols)
    - output_dir: directory to save CSV files
    - train_size: proportion of training samples (default 0.6)
    - val_size: proportion of validation samples (default 0.2)
    - test_size: proportion of test samples (default 0.2)
    - random_state: reproducibility seed
    """
    assert abs(train_size + val_size + test_size - 1.0) < 1e-6,
    assert label_col in df.columns,
    assert label_col in non_feature_cols,

    # First split: train vs temp
    df_train, df_temp = train_test_split(
        df,
        stratify=df[label_col],
        test_size=val_size + test_size,
        random_state=random_state
    )

    # Compute adjusted validation/test split
    val_ratio = val_size / (val_size + test_size)

    # Second split: validation vs test
    df_val, df_test = train_test_split(
        df_temp,
        stratify=df_temp[label_col],
        test_size=1 - val_ratio,
        random_state=random_state
    )

    # Generate features
    feature_cols = [col for col in df.columns if col not in non_feature_cols]
    X_train = df_train[feature_cols]
    X_val = df_val[feature_cols]
    X_test = df_test[feature_cols]

    # Generate labels
    y_train = df_train[[label_col]]
    y_val = df_val[[label_col]]
    y_test = df_test[[label_col]]

    # Metadata
    meta_train = df_train.copy()
    meta_val = df_val.copy()
    meta_test = df_test.copy()

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Save to CSV
    X_train.to_csv(os.path.join(output_dir, "X_train.csv"), index=False)
    X_val.to_csv(os.path.join(output_dir, "X_val.csv"), index=False)
    X_test.to_csv(os.path.join(output_dir, "X_test.csv"), index=False)

    y_train.to_csv(os.path.join(output_dir, "y_train.csv"), index=False)
    y_val.to_csv(os.path.join(output_dir, "y_val.csv"), index=False)
    y_test.to_csv(os.path.join(output_dir, "y_test.csv"), index=False)

    meta_train.to_csv(os.path.join(output_dir, "meta_train.csv"), index=False)
    meta_val.to_csv(os.path.join(output_dir, "meta_val.csv"), index=False)
    meta_test.to_csv(os.path.join(output_dir, "meta_test.csv"), index=False)

    print(f"Data split and saved to '{output_dir}'")

    return (X_train, X_val, X_test), (y_train, y_val, y_test), (meta_train, meta_val, meta_test)

# Dataset clean-up

In [11]:
df = pd.read_excel("ds_FLOWSOM_29092021.xlsx")
df = df.dropna(subset=["IUIS"])
df.shape

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


(399, 89)

In [3]:
# In case I need classes as numbers
classes = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}
for i, row in df.iterrows():
  if str(row["IUIS"]) == "No arguments for lymphoid-PID":
    df.at[i, "IUIS"] = 0
    df.at[i, "IUIS extended"] = 0
  else:
    iuis = str(row["IUIS"])
    index = iuis.find(":")
    pid = iuis[:index]
    df.at[i, "IUIS"] = classes[pid]

# IUIS extended
for i, row in df.iterrows():
  if str(row["IUIS extended"]) == "Unclassified":
    df.at[i, "IUIS extended"] = 0
  else:
    subclass = str(row["IUIS extended"])
    df.at[i, "IUIS extended"] = int(subclass[0])

# create subclassification columns
for i, row in df.iterrows():
  if row["IUIS"] == 1:
    df.at[i, "seq1"] = 1
  else:
    df.at[i, "seq1"] = 0

for i, row in df.iterrows():
  if row["IUIS"] == 0:
    df.at[i, "seq2"] = 0
  else:
    df.at[i, "seq2"] = 1

for i, row in df.iterrows():
  if row["IUIS"] == 0 or row["IUIS"] == 1:
    df.at[i, "seq3"] = 4
  if row["IUIS"] == 2:
    df.at[i, "seq3"] = 0
  if row["IUIS"] == 3:
    if row["IUIS extended"] == 4:
      df.at[i, "seq3"] = 2
    else:
      df.at[i, "seq3"] = 1
  if row["IUIS"] == 4:
    df.at[i, "seq3"] = 3

In [12]:
label_columns = ['IgG (normalized against RV)', 'IGG2 (normalized against RV)',
                 'IGG3 (normalized against RV)','IgM (normalized against RV)',
                 'IGA (normalized against RV)']
df[label_columns] = df[label_columns].fillna('NA')
replacement_dict = {0: 'normal', -1: 'lower', 1: 'higher'}
df[label_columns] = df[label_columns].replace(replacement_dict)

In [13]:
df['Humoral or cellular (B or T-cell mediated)'] = df['Humoral or cellular (B or T-cell mediated)'].str.strip()

In [14]:
to_drop = ['Age (d)', 'labels: lymphoid PID vs HC vc DC vs non-lymphoid PID', 'Subgroep labels', 'Diagnose PID', 'Detailed diagnosis',
           'Humoral or cellular (B or T-cell mediated)', 'Deviations seen in current PIDOT routine appraoch (MG + comparison EuroFlow reference range)',
           'Opmerking', 'Date of birth ', 'Date of sample', 'Unnamed: 69', 'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72', 'Unnamed: 73',
           'Unnamed: 74', 'Unnamed: 75', 'Unnamed: 76', 'Unnamed: 77', 'Unnamed: 78', 'Unnamed: 79', 'Unnamed: 80', 'Unnamed: 81',
           'Unnamed: 82', 'Unnamed: 83', 'Unnamed: 84', 'Unnamed: 85', 'Unnamed: 86', 'Unnamed: 87', 'Unnamed: 88']
df = df.drop(columns=to_drop)

In [15]:
df["% lympho"] = df["% lympho"] /100
df = df.rename(columns={" lympho/µL": "lympho/µL"})
df = df.rename(columns={"EM CD8+ T-cells/µL ": 'EM CD8+ T-cells/µL'})

In [16]:
df.head()

,IUIS,IUIS extended,PCODE,M:F,Age (Y),WBC,Lymfo/µL (Sysmex),IgG,IGG2,IGG3,...,% Effector TD CD4+ T-cells (on CD4 T),% CD8+ T-cells (on T),% naieve CD8+ T-cells (on CD8 T),% CM CD8+ T-cells (on CD8 T),% EM CD8+ T-cells (on CD8 T),% Effector TD 27+ CD8+ T-cells (on CD8 T),% Effector TD CD8+ T-cells (on CD8 T),% DNT TCRgd-T-cells (on T),% TCRgd+ T-cells (on T),% NK cells (on lymfo)
0,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P001,F,12.250000,8720,2280,6,2.07,0.178,...,0.004,0.30,0.67,0.27,0.015,0.031,0.010,0.020,0.060,0.070
1,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P002,M,15.083333,6780,1940,5.9,1.63,0.175,...,0.001,0.27,0.57,0.35,0.031,0.044,0.005,0.013,0.121,0.081
2,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P003,F,17.250000,5950,2040,11.8,3.53,0.253,...,0.000,0.34,0.52,0.37,0.035,0.075,0.013,NaN,0.076,0.089
3,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P004,M,15.750000,3910,1350,10,2.46,0.929,...,0.000,0.43,0.43,0.42,0.141,0.088,0.041,0.025,0.018,0.047
4,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P005,M,15.000000,9200,1860,12.6,2.79,1.27,...,0.020,0.33,0.45,0.01,0.013,0.443,0.080,0.018,0.260,0.153


In [17]:
df.to_csv("ds_FLOWSOM.csv", index=False)

# Subset split

In [26]:
df = pd.read_csv("ds_FLOWSOM.csv")

In [27]:
df = encode_iuis_columns(df)

In [33]:
label_counts = count_label_combinations(df, main_label_col='IUIS', sub_label_col='IUIS extended')
label_counts.to_csv("label_counts.csv", index=False)

In [32]:
(X_train, X_val, X_test), (y_train, y_val, y_test), (meta_train, meta_val, meta_test) = stratified_split_and_save(
    df=df,
    non_feature_cols=['PCODE', 'IUIS', 'IUIS extended', 'seq1', 'seq2', 'seq3'],
    label_col='seq3', # Corrected from ['IUIS'] to 'IUIS'
    output_dir="split"
)

Data split and saved to 'split'
